In [250]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

In [251]:
!pip install pyspark

In [253]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master('local[4]')\
        .appName('Lesson_2')\
        .config('spark.ui.port', '4050')\
        .config('spark.executor.instances', 2)\
        .config('spark.executor.memory', '5g')\
        .config('spark.executor.cores', 2)\
        .getOrCreate()

sc = spark.sparkContext

In [254]:
spark

# Самостоятельная работа


Требуется выяснить:

1. Какое соотношение сторон экрана телефона самое популярное,
2. Плотность пикселей у экрана.
Можно использовать только rdd.

Всего 8 баллов.

# Считывание данных

Данные взяты отсюда: https://www.kaggle.com/iabhishekofficial/mobile-price-classification Скачиваем и копируем в папку с нотебуком

Внутри содержится следующая информация:

* id: ID
* battery_power: Total energy a battery can store in one time (mAh)
* blue: Support bluetooth or not
* clock_speed: Speed at which microprocessor executes instructions
* dual_sim: Support dual sim or not
* fc: Front Camera mega pixels
* four_g: Support 4G or not
* int_memory: Internal Memory (GB)
* m_dep: Mobile Depth (cm)
* mobile_wt: Weight of mobile phone
* n_cores: Number of cores of processor
* pc: Primary Camera mega pixels
* px_height: Pixel Resolution Height
* px_width: Pixel Resolution Width
* ram: Random Access Memory (MB)
* sc_h: Screen Height of mobile (cm)
* sc_w: Screen Width of mobile (cm)
* talk_time: Time that a single battery charge will last
* three_g: Support 3G or not
* touch_screen: Has touch screen or not
* wifi: Support wifi or not

In [618]:
# Вспомогательная функция

def beautiful_print(rdd, num, title):
    print(title)
    for row in rdd.take(num):
        print('\t', row)

In [619]:
train = sc.textFile('./train.csv')
test = sc.textFile('./test.csv')

In [620]:
# Выведите содержимое каждого файла (часть строк) (1 балл)
train.take(5)

['battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range',
 '842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1,1',
 '1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0,2',
 '563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0,2',
 '615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0,2']

In [621]:
test.take(5)

['id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi',
 '1,1043,1,1.8,1,14,0,5,0.1,193,3,16,226,1412,3476,12,7,2,0,1,0',
 '2,841,1,0.5,1,4,1,61,0.8,191,5,12,746,857,3895,6,0,7,1,0,0',
 '3,1807,1,2.8,0,1,0,27,0.9,186,3,4,1270,1366,2396,17,10,10,0,1,1',
 '4,1546,0,0.5,1,18,1,25,0.5,96,8,20,295,1752,3893,10,0,7,1,1,0']

## Преобразуем train и test

In [622]:
train_first_row = train.first()

train = train\
    .filter(lambda row: row != train_first_row)\
    .map(lambda row: [float(el) for el in row.split(',')])

In [623]:
# train.take(3)
beautiful_print(train, 3, 'Train')

Train
	 [842.0, 0.0, 2.2, 0.0, 1.0, 0.0, 7.0, 0.6, 188.0, 2.0, 2.0, 20.0, 756.0, 2549.0, 9.0, 7.0, 19.0, 0.0, 0.0, 1.0, 1.0]
	 [1021.0, 1.0, 0.5, 1.0, 0.0, 1.0, 53.0, 0.7, 136.0, 3.0, 6.0, 905.0, 1988.0, 2631.0, 17.0, 3.0, 7.0, 1.0, 1.0, 0.0, 2.0]
	 [563.0, 1.0, 0.5, 1.0, 2.0, 1.0, 41.0, 0.9, 145.0, 5.0, 6.0, 1263.0, 1716.0, 2603.0, 11.0, 2.0, 9.0, 1.0, 1.0, 0.0, 2.0]


In [624]:
# Преобразуйте test (1 балл)
test_first_row = test.first()

test = test\
    .filter(lambda row: row != test_first_row)\
    .map(lambda row: [float(el) for el in row.split(',')])

In [625]:
# test.take(3)
beautiful_print(test, 3, 'Test')

Test
	 [1.0, 1043.0, 1.0, 1.8, 1.0, 14.0, 0.0, 5.0, 0.1, 193.0, 3.0, 16.0, 226.0, 1412.0, 3476.0, 12.0, 7.0, 2.0, 0.0, 1.0, 0.0]
	 [2.0, 841.0, 1.0, 0.5, 1.0, 4.0, 1.0, 61.0, 0.8, 191.0, 5.0, 12.0, 746.0, 857.0, 3895.0, 6.0, 0.0, 7.0, 1.0, 0.0, 0.0]
	 [3.0, 1807.0, 1.0, 2.8, 0.0, 1.0, 0.0, 27.0, 0.9, 186.0, 3.0, 4.0, 1270.0, 1366.0, 2396.0, 17.0, 10.0, 10.0, 0.0, 1.0, 1.0]


## Объединим train и test
Найти нужную функцию можно [здесь](https://spark.apache.org/docs/3.1.1/api/python/reference/pyspark.html#rdd-apis)

PS: нужно сделать средсвтвами rdd pd.concat([train, test,], axis=0)

In [627]:
train_first_row

'battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range'

In [628]:
test_first_row

'id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi'

In [629]:
# Находим одинаковые колонки в обоих наборах данных
identical_columns = set(train_first_row.split(',')) & set(test_first_row.split(','))

# Вычисляем номера столбцов, которые различаются
train_excess_cols = [x[0] for x in enumerate(train_first_row.split(',')) if x[1] not in identical_columns] 
test_excess_cols = [x[0] for x in enumerate(test_first_row.split(',')) if x[1] not in identical_columns]

# Формируем новый заголовок, сохраняя первоначальный порядок следования столбцов
data_header = [el for el in train_first_row.split(',') if el in identical_columns]
data_header = ','.join(data_header)

In [630]:
# Объедините train и test (2 балла)
data = train.map(lambda row: [x[1] for x in enumerate(row) if x[0] not in train_excess_cols]) \
       .union(test.map(lambda row: [x[1] for x in enumerate(row) if x[0] not in test_excess_cols]))

In [631]:
data_header

'battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi'

In [633]:
beautiful_print(data, 3, 'Data')

Data
	 [842.0, 0.0, 2.2, 0.0, 1.0, 0.0, 7.0, 0.6, 188.0, 2.0, 2.0, 20.0, 756.0, 2549.0, 9.0, 7.0, 19.0, 0.0, 0.0, 1.0]
	 [1021.0, 1.0, 0.5, 1.0, 0.0, 1.0, 53.0, 0.7, 136.0, 3.0, 6.0, 905.0, 1988.0, 2631.0, 17.0, 3.0, 7.0, 1.0, 1.0, 0.0]
	 [563.0, 1.0, 0.5, 1.0, 2.0, 1.0, 41.0, 0.9, 145.0, 5.0, 6.0, 1263.0, 1716.0, 2603.0, 11.0, 2.0, 9.0, 1.0, 1.0, 0.0]


In [634]:
train.count()

2000

In [635]:
test.count()

1000

In [640]:
data.count()

3000

## Рассчитайте соотношение сторон телефона и экрана

In [641]:
# Выведите отсортированное распределение соотношений сторон экрана(1 балла)
# в разрезе широкоформатные или нет (экран широкоформатный, если соотногшение >=16:9)

# Определяем номера столбцов с шириной и высотой экрана
sc_w = [x[0] for x in enumerate(data_header.split(',')) if x[1] == 'sc_w'][0]
sc_h = [x[0] for x in enumerate(data_header.split(',')) if x[1] == 'sc_h'][0]

# Убираем те записи, в которых ширина или высота экрана нулевая
data = data.filter(lambda row: row[sc_w] != 0)
data = data.filter(lambda row: row[sc_h] != 0)

# Производим вычисление сторон. 
# Насколько мне понятно, в случае с телефоном соотношение рассчитывается как отношение высоты к ширине экрана
screen_aspect_ratio = data.map(lambda row: (round(row[sc_h] / row[sc_w], 2), 1)).sortByKey().countByKey()
screen_aspect_ratio

defaultdict(int,
            {1.06: 46,
             1.07: 17,
             1.08: 28,
             1.09: 16,
             1.1: 26,
             1.11: 14,
             1.12: 35,
             1.13: 15,
             1.14: 37,
             1.15: 16,
             1.17: 47,
             1.18: 14,
             1.19: 8,
             1.2: 69,
             1.21: 17,
             1.22: 25,
             1.23: 13,
             1.25: 56,
             1.27: 27,
             1.29: 26,
             1.3: 20,
             1.31: 13,
             1.33: 54,
             1.36: 22,
             1.38: 29,
             1.4: 59,
             1.42: 12,
             1.43: 17,
             1.44: 12,
             1.45: 18,
             1.46: 10,
             1.5: 89,
             1.55: 14,
             1.56: 19,
             1.57: 15,
             1.58: 10,
             1.6: 32,
             1.62: 8,
             1.64: 12,
             1.67: 68,
             1.7: 16,
             1.71: 26,
             1.73: 8,
    

In [642]:
# Сделаем второй словарь, в котором оставим только широкоформатные модели (ключ >= 1.78)
# Отсортируем соотношения сторон по количеству моделей (по значению)
widescreen = {x[0]: x[1] for x in screen_aspect_ratio.items() if x[0] >= 1.78}
widescreen = sorted(widescreen.items(), key=lambda item: item[1], reverse=True)
widescreen

[(2.0, 123),
 (3.0, 100),
 (6.0, 70),
 (5.0, 68),
 (2.5, 66),
 (4.0, 50),
 (2.33, 46),
 (7.0, 43),
 (4.5, 37),
 (3.5, 35),
 (2.25, 34),
 (8.0, 34),
 (1.8, 33),
 (9.0, 31),
 (2.67, 29),
 (10.0, 26),
 (2.2, 21),
 (2.17, 20),
 (2.43, 20),
 (5.5, 19),
 (11.0, 19),
 (14.0, 19),
 (2.83, 18),
 (4.25, 18),
 (17.0, 18),
 (1.86, 17),
 (2.12, 17),
 (2.6, 17),
 (4.33, 17),
 (2.57, 16),
 (3.25, 16),
 (12.0, 16),
 (1.88, 15),
 (1.9, 15),
 (3.2, 15),
 (6.33, 15),
 (8.5, 15),
 (3.33, 14),
 (3.67, 14),
 (4.67, 14),
 (5.67, 14),
 (7.5, 14),
 (13.0, 14),
 (16.0, 14),
 (1.83, 13),
 (1.89, 13),
 (2.14, 13),
 (2.4, 13),
 (5.33, 13),
 (2.29, 12),
 (2.75, 12),
 (2.8, 12),
 (6.5, 12),
 (2.38, 11),
 (3.75, 11),
 (15.0, 11),
 (3.4, 10),
 (3.6, 10),
 (19.0, 10),
 (2.11, 9),
 (3.17, 9),
 (9.5, 9),
 (18.0, 9),
 (1.78, 8),
 (4.75, 8),
 (2.71, 7),
 (3.8, 4)]

In [643]:
# Выведите отсортированное распределение плотности пикселей (1 балла)
# точек на дюйм

# Определяем номера столбцов с шириной и высотой экрана
sc_w = [x[0] for x in enumerate(data_header.split(',')) if x[1] == 'sc_w'][0]
sc_h = [x[0] for x in enumerate(data_header.split(',')) if x[1] == 'sc_h'][0]

# Определяем номера столбцов с пикселями
px_width = [x[0] for x in enumerate(data_header.split(',')) if x[1] == 'px_width'][0]
px_height = [x[0] for x in enumerate(data_header.split(',')) if x[1] == 'px_height'][0]

# Удаляем строки с нулевыми данными
data = data.filter(lambda row: row[sc_w] != 0)
data = data.filter(lambda row: row[sc_h] != 0)

data = data.filter(lambda row: row[px_width] != 0)
data = data.filter(lambda row: row[px_height] != 0)

# Рассчитываем PPI
PPI = data.map(lambda row: \
                          (\
                            round( \
                              ((row[px_width] ** 2 + row[px_height] ** 2) ** 0.5) / \
                              (((row[sc_w] ** 2 + row[sc_h] ** 2) ** 0.5) * 0.3937) \
                                    , 0) \
                           , 1) \
              ).sortByKey().countByKey()

PPI

defaultdict(int,
            {56.0: 1,
             61.0: 3,
             63.0: 1,
             64.0: 1,
             65.0: 1,
             67.0: 2,
             68.0: 1,
             69.0: 1,
             70.0: 1,
             71.0: 1,
             72.0: 2,
             73.0: 2,
             74.0: 2,
             76.0: 2,
             77.0: 1,
             78.0: 2,
             79.0: 6,
             80.0: 5,
             81.0: 3,
             82.0: 6,
             83.0: 8,
             84.0: 6,
             85.0: 4,
             86.0: 3,
             87.0: 8,
             88.0: 5,
             89.0: 5,
             90.0: 4,
             91.0: 3,
             92.0: 5,
             93.0: 6,
             94.0: 3,
             95.0: 4,
             96.0: 7,
             97.0: 5,
             98.0: 2,
             99.0: 5,
             100.0: 5,
             101.0: 4,
             102.0: 7,
             103.0: 6,
             104.0: 3,
             105.0: 6,
             106.0: 3,
        

In [644]:
# Проверяем сколько в объединенном наборе данных осталось строчек, после того как удалили строчки с нулевыми данными.
data.count()

2705

In [275]:
data_header

'battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi'

# JOIN
Повторите вышеописанное задание с помощью одной из функций ниже (отдельно рассчитайте для train и test, затем объедините результат)

Пример для двух RDDs: (rdd = {(1, 2), (3, 4), (3, 6)} other = {(3, 9)})

Имя функции |	Purpose |	Example |	Result
------------- |	------- |	------- |	------
subtractByKey |Remove elements with a key present in the other RDD.| rdd.subtractByKey(other) | {(1, 2)}
join | Perform an inner join between two RDDs. | rdd.join(other) | {(3, (4, 9)), (3, (6, 9))}
rightOuterJoin | Perform a join between two RDDs where the key must be present in the first RDD. | rdd.rightOuterJoin(other) | {(3,(Some(4),9)), (3,(Some(6),9))}leftOuterJoin | Perform a join between two RDDs where the key must be present in the other RDD. | rdd.leftOuterJoin(other) | {(1,(2,None)), (3,(4,Some(9))), (3,(6,Some(9)))}
cogroup | Group data from both RDDs sharing the same key. | rdd.cogroup(other) | {(1,([2],[])), (3,([4, 6],[9]))}


In [645]:
train_first_row

'battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range'

In [646]:
test_first_row

'id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi'

In [647]:
train.count()

2000

In [648]:
test.count()

1000

In [649]:
# Подготавливаем для работы train

# Определяем номера столбцов с шириной и высотой экрана
tr_sc_w = [x[0] for x in enumerate(train_first_row.split(',')) if x[1] == 'sc_w'][0]
tr_sc_h = [x[0] for x in enumerate(train_first_row.split(',')) if x[1] == 'sc_h'][0]

# Определяем номера столбцов с пикселями
tr_px_width = [x[0] for x in enumerate(train_first_row.split(',')) if x[1] == 'px_width'][0]
tr_px_height = [x[0] for x in enumerate(train_first_row.split(',')) if x[1] == 'px_height'][0]

# Удаляем строки с нулевыми данными
jtrain = train.filter(lambda row: row[tr_sc_w] != 0)
jtrain = jtrain.filter(lambda row: row[tr_sc_h] != 0)
jtrain = jtrain.filter(lambda row: row[tr_px_width] != 0)
jtrain = jtrain.filter(lambda row: row[tr_px_height] != 0)

# Подготавливаем для работы test

# Определяем номера столбцов с шириной и высотой экрана
te_sc_w = [x[0] for x in enumerate(test_first_row.split(',')) if x[1] == 'sc_w'][0]
te_sc_h = [x[0] for x in enumerate(test_first_row.split(',')) if x[1] == 'sc_h'][0]

# Определяем номера столбцов с пикселями
te_px_width = [x[0] for x in enumerate(test_first_row.split(',')) if x[1] == 'px_width'][0]
te_px_height = [x[0] for x in enumerate(test_first_row.split(',')) if x[1] == 'px_height'][0]

# Удаляем строки с нулевыми данными
jtest = test.filter(lambda row: row[te_sc_w] != 0)
jtest = jtest.filter(lambda row: row[te_sc_h] != 0)
jtest = jtest.filter(lambda row: row[te_px_width] != 0)
jtest = jtest.filter(lambda row: row[te_px_height] != 0)

In [650]:
jtrain.count()

1819

In [651]:
jtest.count()

886

In [652]:
# Вычисляем соотношение сторон для train и test
tr_screen_aspect_ratio = jtrain.map(lambda row: (round(row[tr_sc_h] / row[tr_sc_w], 2), 1)).sortByKey().countByKey()
te_screen_aspect_ratio = jtest.map(lambda row: (round(row[te_sc_h] / row[te_sc_w], 2), 1)).sortByKey().countByKey()

# Вычисляем PPI для train и test
tr_PPI = jtrain.map(lambda row: \
                          (\
                            round( \
                              ((row[tr_px_width] ** 2 + row[tr_px_height] ** 2) ** 0.5) / \
                              (((row[tr_sc_w] ** 2 + row[tr_sc_h] ** 2) ** 0.5) * 0.3937) \
                                    , 0) \
                           , 1) \
              ).sortByKey().countByKey()

te_PPI = jtest.map(lambda row: \
                          (\
                            round( \
                              ((row[te_px_width] ** 2 + row[te_px_height] ** 2) ** 0.5) / \
                              (((row[te_sc_w] ** 2 + row[te_sc_h] ** 2) ** 0.5) * 0.3937) \
                                    , 0) \
                           , 1) \
              ).sortByKey().countByKey()

In [653]:
# Создаем объекты RDD, на основании полученных данных о соотношении сторон и PPI
train_subtract = sc.parallelize(tr_screen_aspect_ratio.items())
test_subtract = sc.parallelize(te_screen_aspect_ratio.items())

train_join = sc.parallelize(tr_PPI.items())
test_join = sc.parallelize(te_PPI.items())

In [654]:
train_subtract.subtractByKey(test_subtract).collect()

[(1.09, 16)]

In [655]:
train_join.join(test_join).sortByKey().collect()

[(61.0, (1, 2)),
 (73.0, (1, 1)),
 (74.0, (1, 1)),
 (76.0, (1, 1)),
 (79.0, (4, 2)),
 (80.0, (1, 4)),
 (82.0, (4, 2)),
 (83.0, (4, 4)),
 (84.0, (5, 1)),
 (85.0, (3, 1)),
 (86.0, (2, 1)),
 (87.0, (4, 4)),
 (88.0, (2, 3)),
 (89.0, (2, 3)),
 (90.0, (3, 1)),
 (91.0, (2, 1)),
 (92.0, (3, 2)),
 (93.0, (2, 4)),
 (94.0, (2, 1)),
 (95.0, (3, 1)),
 (97.0, (2, 3)),
 (98.0, (1, 1)),
 (99.0, (4, 1)),
 (100.0, (3, 2)),
 (102.0, (5, 2)),
 (103.0, (4, 2)),
 (105.0, (5, 1)),
 (106.0, (2, 1)),
 (107.0, (4, 1)),
 (108.0, (1, 1)),
 (109.0, (3, 3)),
 (110.0, (5, 3)),
 (111.0, (3, 2)),
 (112.0, (3, 5)),
 (113.0, (2, 1)),
 (114.0, (4, 3)),
 (115.0, (2, 3)),
 (116.0, (5, 1)),
 (117.0, (5, 3)),
 (118.0, (5, 3)),
 (120.0, (4, 3)),
 (121.0, (6, 5)),
 (122.0, (11, 2)),
 (123.0, (4, 2)),
 (124.0, (4, 3)),
 (126.0, (4, 2)),
 (127.0, (6, 2)),
 (128.0, (9, 3)),
 (129.0, (2, 3)),
 (130.0, (1, 1)),
 (131.0, (3, 1)),
 (132.0, (8, 2)),
 (133.0, (7, 1)),
 (134.0, (3, 1)),
 (136.0, (6, 2)),
 (137.0, (8, 4)),
 (138.0, (7, 6

# DataFrame
Теперь мы знаем про Dataframe. Нужно сделать практически всё то же самое, но используя датафрейм.

In [656]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, FloatType
# Считываем и объединяем данные (1 балл)
# Приведите все данные к правильному типу, либо считайе сразу верно (1 балл)
# Создаём колонки с соотношением сторон и плотностью пикселей (1 балл)

# Считываем данные сразу с правильными типами (inferSchema). 
df_train = spark.read.csv('./train.csv',header = True, inferSchema = True)
df_test = spark.read.csv('./test.csv',header = True, inferSchema = True)

In [657]:
df_train.printSchema()

root
 |-- battery_power: integer (nullable = true)
 |-- blue: integer (nullable = true)
 |-- clock_speed: double (nullable = true)
 |-- dual_sim: integer (nullable = true)
 |-- fc: integer (nullable = true)
 |-- four_g: integer (nullable = true)
 |-- int_memory: integer (nullable = true)
 |-- m_dep: double (nullable = true)
 |-- mobile_wt: integer (nullable = true)
 |-- n_cores: integer (nullable = true)
 |-- pc: integer (nullable = true)
 |-- px_height: integer (nullable = true)
 |-- px_width: integer (nullable = true)
 |-- ram: integer (nullable = true)
 |-- sc_h: integer (nullable = true)
 |-- sc_w: integer (nullable = true)
 |-- talk_time: integer (nullable = true)
 |-- three_g: integer (nullable = true)
 |-- touch_screen: integer (nullable = true)
 |-- wifi: integer (nullable = true)
 |-- price_range: integer (nullable = true)



In [658]:
df_test.printSchema()

root
 |-- id: integer (nullable = true)
 |-- battery_power: integer (nullable = true)
 |-- blue: integer (nullable = true)
 |-- clock_speed: double (nullable = true)
 |-- dual_sim: integer (nullable = true)
 |-- fc: integer (nullable = true)
 |-- four_g: integer (nullable = true)
 |-- int_memory: integer (nullable = true)
 |-- m_dep: double (nullable = true)
 |-- mobile_wt: integer (nullable = true)
 |-- n_cores: integer (nullable = true)
 |-- pc: integer (nullable = true)
 |-- px_height: integer (nullable = true)
 |-- px_width: integer (nullable = true)
 |-- ram: integer (nullable = true)
 |-- sc_h: integer (nullable = true)
 |-- sc_w: integer (nullable = true)
 |-- talk_time: integer (nullable = true)
 |-- three_g: integer (nullable = true)
 |-- touch_screen: integer (nullable = true)
 |-- wifi: integer (nullable = true)



In [659]:
# Находим общие столбцы в обоих файлах
ident_columns = set(df_train.columns) & set(df_test.columns)

# Удаляем различающиеся столбцы из обоих dataframe'ов
df_train = df_train.drop(*(set(df_train.columns) - ident_columns))
df_test = df_test.drop(*(set(df_test.columns) - ident_columns))

# Объединяем данные
df_data = df_train.union(df_test)

In [660]:
df_train.count()

2000

In [661]:
df_test.count()

1000

In [662]:
df_data.count()

3000

In [663]:
df_data.printSchema()

root
 |-- battery_power: integer (nullable = true)
 |-- blue: integer (nullable = true)
 |-- clock_speed: double (nullable = true)
 |-- dual_sim: integer (nullable = true)
 |-- fc: integer (nullable = true)
 |-- four_g: integer (nullable = true)
 |-- int_memory: integer (nullable = true)
 |-- m_dep: double (nullable = true)
 |-- mobile_wt: integer (nullable = true)
 |-- n_cores: integer (nullable = true)
 |-- pc: integer (nullable = true)
 |-- px_height: integer (nullable = true)
 |-- px_width: integer (nullable = true)
 |-- ram: integer (nullable = true)
 |-- sc_h: integer (nullable = true)
 |-- sc_w: integer (nullable = true)
 |-- talk_time: integer (nullable = true)
 |-- three_g: integer (nullable = true)
 |-- touch_screen: integer (nullable = true)
 |-- wifi: integer (nullable = true)



In [664]:
# Создаём колонки с соотношением сторон и плотностью пикселей

# Отсеиваем строки с нулевыми данными 
df_data = df_data.select('*').where('sc_h != 0 and sc_w != 0 and px_height != 0 and px_width != 0')

# Добавляем столбец с соотношением сторон
df_data = df_data.withColumn('screen_ratio', F.round((F.col('sc_h')/F.col('sc_w')).cast(FloatType()), 2))

# Создаём колонки с соотношением сторон и плотностью пикселей
df_data = df_data.withColumn('ppi', F.round( \
                                            (F.sqrt((F.pow(F.col('px_width'), 2) + F.pow(F.col('px_height'), 2))) / \
                                            (F.sqrt(F.pow(F.col('sc_w'), 2) + F.pow(F.col('sc_h'), 2)) * 0.3937)) \
                                            .cast(FloatType()), 0))

In [665]:
df_data.count()

2705

In [666]:
df_data.printSchema()

root
 |-- battery_power: integer (nullable = true)
 |-- blue: integer (nullable = true)
 |-- clock_speed: double (nullable = true)
 |-- dual_sim: integer (nullable = true)
 |-- fc: integer (nullable = true)
 |-- four_g: integer (nullable = true)
 |-- int_memory: integer (nullable = true)
 |-- m_dep: double (nullable = true)
 |-- mobile_wt: integer (nullable = true)
 |-- n_cores: integer (nullable = true)
 |-- pc: integer (nullable = true)
 |-- px_height: integer (nullable = true)
 |-- px_width: integer (nullable = true)
 |-- ram: integer (nullable = true)
 |-- sc_h: integer (nullable = true)
 |-- sc_w: integer (nullable = true)
 |-- talk_time: integer (nullable = true)
 |-- three_g: integer (nullable = true)
 |-- touch_screen: integer (nullable = true)
 |-- wifi: integer (nullable = true)
 |-- screen_ratio: float (nullable = true)
 |-- ppi: float (nullable = true)



In [667]:
df_data.select('battery_power', 'sc_h', 'sc_w', 'screen_ratio', 'px_width', 'px_height', 'ppi').show(10)

+-------------+----+----+------------+--------+---------+-----+
|battery_power|sc_h|sc_w|screen_ratio|px_width|px_height|  ppi|
+-------------+----+----+------------+--------+---------+-----+
|          842|   9|   7|        1.29|     756|       20|168.0|
|         1021|  17|   3|        5.67|    1988|      905|321.0|
|          563|  11|   2|         5.5|    1716|     1263|484.0|
|          615|  16|   8|         2.0|    1786|     1216|307.0|
|         1821|   8|   2|         4.0|    1212|     1208|527.0|
|         1859|  17|   1|        17.0|    1654|     1004|289.0|
|         1821|  13|   8|        1.63|    1018|      381|181.0|
|         1954|  16|   3|        5.33|    1149|      512|196.0|
|         1445|  17|   1|        17.0|     836|      386|137.0|
|          509|  19|  10|         1.9|    1224|     1137|198.0|
+-------------+----+----+------------+--------+---------+-----+
only showing top 10 rows



In [668]:
# Для демонстрации выводим только широкоформатные модели
df_data.select('screen_ratio') \
       .groupBy('screen_ratio').count() \
       .orderBy('screen_ratio').filter(F.col('screen_ratio') >= 1.77).show(10)

+------------+-----+
|screen_ratio|count|
+------------+-----+
|        1.78|    8|
|         1.8|   33|
|        1.83|   13|
|        1.86|   17|
|        1.88|   15|
|        1.89|   13|
|         1.9|   15|
|         2.0|  123|
|        2.11|    9|
|        2.13|   17|
+------------+-----+
only showing top 10 rows



In [669]:
# Для демонстрации выводим ppi в отсортированном виде
df_data.select('ppi').groupBy('ppi').count().orderBy('ppi').show(10)

+----+-----+
| ppi|count|
+----+-----+
|56.0|    1|
|61.0|    3|
|63.0|    1|
|64.0|    1|
|65.0|    1|
|67.0|    2|
|68.0|    1|
|69.0|    1|
|70.0|    1|
|71.0|    1|
+----+-----+
only showing top 10 rows



In [670]:
# Сохраните результат в csv sep=';', encoding='cp1251'
# с колонками id, плотность пикселей и временем разговора в формате "1day 1hour 1minute"
# 2 балла

# !!!!! ЗАДАНИЕ С ВРЕМЕНЕМ РАЗГОВОРА НЕ ПОНЯЛ !!!!!
# НУЖНЫЙ СТОЛБЕЦ НАШЕЛ (talk_time). НЕ ПОНЯЛ СМЫСЛА ЗАДАНИЯ
# ПОСЧИТАЛ, ЧТО ЗАДАНИЕ НА ИСПОЛЬЗОВАНИЕ ФУНКЦИИ КОНКАТЕНАЦИИ.
# СКОРЕЕ ВСЕГО, ЗАДАНИЕ ПОНЯЛ НЕПРАВИЛЬНО. НО С САМОЙ ФУНКЦИЕЙ РАЗОБРАЛСЯ.

from pyspark.sql import Window

# Добавляем столбец с id. Так как выводится в итоговый файл ppi, то решил к нему id привязать.
window = Window.orderBy('ppi')
df_data_final = df_data.withColumn('id', F.row_number().over(window))
# Затем добавил столбец с временем разговора.
# В итоговый набор данных добавил поля id, ppi, talk_time, time (преобразованный согласно заданию talk_time)
df_data_final = df_data_final.select('id', 'ppi', 'talk_time') \
                             .withColumn('time', F.concat(F.lit('1'), F.concat_ws(' 1', \
                                                                        (F.col('talk_time') /  24).cast('int'), \
                                                                        (F.col('talk_time') % 24), \
                                                                         F.lit('0'))))

df_data_final.write.csv(path = './result.csv', mode = 'overwrite', sep = ';', encoding = 'cp1251', header = True)

21/08/16 07:55:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [671]:
df_data_final.select('*').count()

2705

In [673]:
df_data_final.columns

['id', 'ppi', 'talk_time', 'time']